In [1]:
import cv2
import numpy as np
from PIL import Image,ImageTk
import imutils
import io
import mysql.connector as mc
import datetime
import easyocr
from tkinter import *
from tkinter import ttk

In [2]:
def crop_image():
    img = cv2.imread(r'C:\Users\Adhavan.G\OneDrive\Desktop\sem 3\image4.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(bfilter, 30, 200)
    #plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0,255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)

    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)
    return result[0][1]

In [3]:
def binary():
    with open(r'C:\Users\Adhavan.G\OneDrive\Desktop\sem 3\image4.jpg','rb') as f:
        res=f.read()
    return res


In [4]:
date=datetime.date.today()
now = datetime.datetime.now()
time = now.strftime("%H:%M:%S")
text=crop_image()
image=binary()
print(text)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\users\adhavan.g\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


H982 FKL


In [5]:
mydb=mc.connect(host='127.0.0.1',user='root',port=3306,password='adhav1405',database='vehicle')
data=mydb.cursor()
data.execute("insert into vehicle(DATE,TIME,TEXT,IMAGE) value(%s,%s,%s,%s)",(date,time,text,image))

In [6]:
def get_value():
    data.execute("select * from vehicle where {}=%s".format(drop.get()),(entry1.get(),))
    count=1
    for i in data:
        for j in range(3):
            listbox.insert(END,i[j]+"\n")
        def convert_image(image):
            with open(r"D:\INTERN\python exercise\vehicle track images\image"+str(count)+'.jpg','wb') as f:
                f.write(i[3])
            img1=Image.open(r"D:\INTERN\python exercise\vehicle track images\image"+str(count)+'.jpg')
            m=img1.resize((300,205),Image.ANTIALIAS)
            myimage=ImageTk.PhotoImage(m)
            listbox.image_create(END,image=myimage)
            listbox.insert(END,'\n')
        convert_image(i[3])
        count+=1

In [7]:
root=Tk()
root.title("VEHICLE TRACKING")
root.geometry("1000x600")
drop=ttk.Combobox(root,value=['date','text','time'])
drop.pack()

entry1=Entry(root,width=30)
entry1.pack()

button2=Button(root,text='Get value',command=get_value)
button2.pack()

my_frame=Frame(root)
scrollbar1=Scrollbar(my_frame,orient=VERTICAL)
scrollbar2=Scrollbar(my_frame,orient=HORIZONTAL)
listbox=Text(my_frame,width=200,height=40,yscrollcommand=scrollbar1.set)
scrollbar1.config(command=listbox.yview)
scrollbar1.pack(side='right',fill=Y)
scrollbar2.config(command=listbox.xview)
scrollbar2.pack(side='bottom',fill=X)
listbox.pack(pady=15)
my_frame.pack()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\adhavan.g\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\Adhavan.G\AppData\Local\Temp/ipykernel_13204/2774620668.py", line 15, in get_value
    convert_image(i[3])
  File "C:\Users\Adhavan.G\AppData\Local\Temp/ipykernel_13204/2774620668.py", line 10, in convert_image
    img1=Image.open(r"D:\INTERN\python exercise\vehicle track images\image"+str(count)+'.jpg')
AttributeError: type object 'Image' has no attribute 'open'


In [9]:
import os
for file in os.listdir(r"D:\INTERN\python exercise\vehicle track images"):
    os.remove(r"D:\INTERN\python exercise\vehicle track images"+'\\'+file)